<a href="https://colab.research.google.com/github/jenmusdat/Project3_BeerQuality/blob/main/Analysis/training_feel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
# Start a SparkSession
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("word2vec").getOrCreate()

In [4]:
MAX_MEMORY = "5g"

spark = SparkSession \
    .builder \
    .appName("word2vec") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config("spark.driver.maxResultSize", MAX_MEMORY)\
    .getOrCreate()

In [5]:
# Import needed libs
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.ml.feature import Word2Vec, Word2VecModel
import string


In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="/content/gdrive/MyDrive/Project3_BeerQuality/Resources/reviews_beer_brewery.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reviews_beer_brewery.csv"), header=True)


In [7]:
# remove alpha characters from data set on review numerics
from pyspark.sql.functions import regexp_extract, col
pattern = r'^\d+.\d+$'
df = df.filter(regexp_extract(col('look'), pattern, 0) != '')

In [8]:
df = df.filter(regexp_extract(col('smell'), pattern, 0) != '')
df = df.filter(regexp_extract(col('taste'), pattern, 0) != '')
df = df.filter(regexp_extract(col('overall'), pattern, 0) != '')
df = df.filter(regexp_extract(col('score'), pattern, 0) != '')
df = df.filter(regexp_extract(col('feel'), pattern, 0) != '')

In [9]:
# Next 6 lines casts to floats all the review ratings
df = df.withColumn('look_numeric', df['look'].cast('float'))

In [10]:
df = df.withColumn('smell_numeric', df['smell'].cast('float'))

In [11]:
df = df.withColumn('taste_numeric', df['taste'].cast('float'))

In [12]:
df = df.withColumn('feel_numeric', df['feel'].cast('float'))

In [13]:
df = df.withColumn('overall_numeric', df['overall'].cast('float'))

In [14]:
df = df.withColumn('score_numeric', df['score'].cast('float'))

In [15]:
# Confirm data types
df.printSchema

<bound method DataFrame.printSchema of DataFrame[beer_id: string, username: string, date: string, text: string, look: string, smell: string, taste: string, feel: string, overall: string, score: string, beer_name: string, review_state: string, style: string, availability: string, abv: string, brewery_id: string, brewery_name: string, brewery_city: string, brewery_state: string, brewery_types: string, look_numeric: float, smell_numeric: float, taste_numeric: float, feel_numeric: float, overall_numeric: float, score_numeric: float]>

In [16]:
# functions for manipulating review text into a format we can tokenize
def remove_punctuation(txt):
  txt = txt.strip('\xa0\xa0 ')
  return "".join(l if l not in string.punctuation else "" for l in txt)

def remove_space (text):
  if text:
    return text.replace(' ', '')
  else:
    return 'Unknown'
  

In [17]:
# define the udfs for the data
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, udf

remove_punctuation_udf = udf(remove_punctuation, StringType())
remove_punctuation_udf

remove_space_udf = udf(remove_space, StringType())
remove_space_udf

<function __main__.remove_space>

In [18]:
# clean the beer name by removing spaces (will be combined with text)
beer_df = df.withColumn('beer_clean', remove_space_udf(col('beer_name')))
# beer_df.show()

In [19]:
# clean the style name by removing spaces (will be combined with text)
style_df = beer_df.withColumn('style_clean', remove_space_udf(col('style')))
# style_df.show()

In [20]:
# clean the text by removing punctuation
text_df = style_df.withColumn("clean_text", remove_punctuation_udf(col("text")))
text_df.show()

+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+------------+-------------+-------------+------------+-------------+-------------+------------+---------------+-------------+--------------------+--------------------+--------------------+
|beer_id|        username|      date|                text|look|smell|taste|feel|overall|score|           beer_name|review_state|               style|        availability| abv|brewery_id|        brewery_name|brewery_city|brewery_state|brewery_types|look_numeric|smell_numeric|taste_numeric|feel_numeric|overall_numeric|score_numeric|          beer_clean|         style_clean|          clean_text|
+-------+----------------+----------+--------------------+----+-----+-----+----+-------+-----+--------------------+------------+--------------------+--------------------+----+----------+--------------------+-

In [21]:
# Combine beer_name, style, and review text into a Final_text column
from pyspark.sql.functions import concat_ws

combined_df = text_df.withColumn('Final_text', concat_ws(' ',text_df.beer_clean, text_df.style_clean, text_df.clean_text))
combined_df.select('text', 'Final_text').show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [64]:
# Setup and execute Tokenizer and Stop words pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml import Pipeline

# "Creating pipeline..."
tokenizer = Tokenizer(inputCol="Final_text", outputCol="token_text")
stopremove = StopWordsRemover(inputCol='token_text', outputCol='features')

pipeline = Pipeline(stages=[tokenizer, stopremove])

# "Training model..."
pipeline_stg = pipeline.fit(combined_df)
final_df = pipeline_stg.transform(combined_df)


In [ ]:
# final_df.show()

In [65]:
final_df = final_df.select('feel_numeric','features')

In [66]:
final_df.show()

+------------+--------------------+
|feel_numeric|            features|
+------------+--------------------+
|        4.25|[motorbreathimper...|
|         3.5|[paybackpilsner, ...|
|        3.75|[paybackpilsner, ...|
|         3.5|[paybackpilsner, ...|
|         3.5|[paybackpilsner, ...|
|         3.5|[paybackpilsner, ...|
|         3.5|[paybackpilsner, ...|
|         4.0|[paybackpilsner, ...|
|         3.5|[paybackpilsner, ...|
|        3.75|[paybackpilsner, ...|
|         3.5|[hellhammer, ryeb...|
|        3.75|[hellhammer, ryeb...|
|         3.0|[vanillaporter, a...|
|         4.0|[vanillaporter, a...|
|         3.0|[vanillaporter, a...|
|         3.0|[vanillaporter, a...|
|         2.5|[vanillaporter, a...|
|         4.0|[vanillaporter, a...|
|        3.75|[vanillaporter, a...|
|        3.75|[vanillaporter, a...|
+------------+--------------------+
only showing top 20 rows



In [67]:
# Read the model from disk to confirm
w2v_loaded_model = Word2VecModel.load('/content/gdrive/MyDrive/Project3_BeerQuality/review.md')

In [68]:
# check to see if synonym is consistent with loaded model
w2v_loaded_model.findSynonymsArray("light", 5)

[('mild', 0.6426884531974792),
 ('faint', 0.6223998665809631),
 ('slight', 0.6191715002059937),
 ('mostly', 0.6169756650924683),
 ('medium', 0.6095581650733948)]

In [69]:
# Transform the mode
w2v_model_df = w2v_loaded_model.transform(final_df)

In [70]:
w2v_model_df.show()

+------------+--------------------+--------------------+
|feel_numeric|            features|               model|
+------------+--------------------+--------------------+
|        4.25|[motorbreathimper...|[0.02950784611367...|
|         3.5|[paybackpilsner, ...|[-0.0872463285923...|
|        3.75|[paybackpilsner, ...|[0.01790586973612...|
|         3.5|[paybackpilsner, ...|[-0.1276244542612...|
|         3.5|[paybackpilsner, ...|[-0.0131208907440...|
|         3.5|[paybackpilsner, ...|[-0.0353247860187...|
|         3.5|[paybackpilsner, ...|[-0.0606238029503...|
|         4.0|[paybackpilsner, ...|[0.00897214795847...|
|         3.5|[paybackpilsner, ...|[0.02342551416502...|
|        3.75|[paybackpilsner, ...|[-0.0209101041158...|
|         3.5|[hellhammer, ryeb...|[0.01432307037573...|
|        3.75|[hellhammer, ryeb...|[-0.1081325657665...|
|         3.0|[vanillaporter, a...|[-0.1101182210924...|
|         4.0|[vanillaporter, a...|[-0.0197092709394...|
|         3.0|[vanillaporter, a

In [71]:
w2v_model_df = w2v_model_df.select('feel_numeric', 'model')

In [72]:
# Prepare the hyperparameters
from pyspark.ml.regression import LinearRegression, LinearRegressionModel
lin_model = LinearRegression(
   maxIter=5,
   regParam=0.3,
   featuresCol="model",
   labelCol="feel_numeric"
)

In [73]:
w2v_model_df.cache()
w2v_model_df.show()

+------------+--------------------+
|feel_numeric|               model|
+------------+--------------------+
|        4.25|[0.02950784611367...|
|         3.5|[-0.0872463285923...|
|        3.75|[0.01790586973612...|
|         3.5|[-0.1276244542612...|
|         3.5|[-0.0131208907440...|
|         3.5|[-0.0353247860187...|
|         3.5|[-0.0606238029503...|
|         4.0|[0.00897214795847...|
|         3.5|[0.02342551416502...|
|        3.75|[-0.0209101041158...|
|         3.5|[0.01432307037573...|
|        3.75|[-0.1081325657665...|
|         3.0|[-0.1101182210924...|
|         4.0|[-0.0197092709394...|
|         3.0|[-0.0049533498369...|
|         3.0|[0.02202844081446...|
|         2.5|[-0.0874559852428...|
|         4.0|[-0.0728758174786...|
|        3.75|[-0.1125914826324...|
|        3.75|[-0.1178625054068...|
+------------+--------------------+
only showing top 20 rows



In [74]:
# split training and testing for the model (for temporary make the training set small as processing takes time)
training, testing = w2v_model_df.randomSplit([0.60, 0.40])

In [75]:
training.show()

+------------+--------------------+
|feel_numeric|               model|
+------------+--------------------+
|         1.0|[-0.1452391079316...|
|         1.0|[-0.1360762032201...|
|         1.0|[-0.1245115498701...|
|         1.0|[-0.1112089413319...|
|         1.0|[-0.1096023616691...|
|         1.0|[-0.1067826888812...|
|         1.0|[-0.0992016121461...|
|         1.0|[-0.0905129771966...|
|         1.0|[-0.0898339535608...|
|         1.0|[-0.0831069555772...|
|         1.0|[-0.0829146506434...|
|         1.0|[-0.0806717465166...|
|         1.0|[-0.0735591155427...|
|         1.0|[-0.0698463073019...|
|         1.0|[-0.0685070913528...|
|         1.0|[-0.0677027806639...|
|         1.0|[-0.0668491764137...|
|         1.0|[-0.0643640787910...|
|         1.0|[-0.0561385970594...|
|         1.0|[-0.0556136497510...|
+------------+--------------------+
only showing top 20 rows



In [76]:
training_pandas_df = training.toPandas()

In [77]:
training_pandas_df.isnull().sum()

feel_numeric    0
model           0
dtype: int64

In [78]:
training_pandas_df = training_pandas_df.dropna()

In [79]:
training_pandas_df.head()

,feel_numeric,model
0,1.0,"[-0.14523910793165368, 0.12042828451505759, -0..."
1,1.0,"[-0.13607620322019667, 0.1249764144194086, -0...."
2,1.0,"[-0.12451154987017313, -0.30530258764823276, -..."
3,1.0,"[-0.11120894133194309, 0.08106049458476026, -0..."
4,1.0,"[-0.10960236166914304, -0.031228498804072538, ..."


In [80]:
training_pandas_df.dtypes

feel_numeric    float32
model            object
dtype: object

In [81]:
from sklearn.linear_model import LinearRegression
pandas_lin_model = LinearRegression()

In [82]:
lin_feel_train_model = pandas_lin_model.fit(training_pandas_df['model'].to_list(),training_pandas_df['feel_numeric'])

In [83]:
lin_feel_train_model.score(training_pandas_df['model'].to_list(), training_pandas_df['feel_numeric'])

0.3484855955762809

In [84]:
lin_feel_train_model.coef_

array([ 0.29686585,  1.06724775,  0.89359459, -0.13957956,  0.35333396,
        1.17521239,  0.80825058,  0.99195495, -0.07610251, -0.18667602,
        0.06374701, -0.24617509,  0.91204411,  0.5045303 ,  0.64601178,
        0.62258019, -0.72159604,  0.49600582, -0.22782096, -0.64009236,
       -0.11182773, -0.52797132,  0.16537613, -0.40856393,  0.20891129,
       -0.26709142, -0.25661207, -0.71021999, -0.78766279,  0.96976033,
        0.82720538, -0.2801861 ,  0.28527831, -0.12806586,  0.19365689,
       -0.88044621, -0.59704157,  0.25263249,  0.19122042, -0.25611425,
        0.19346351, -0.89360941,  0.67916329, -0.79667508,  0.06880932,
       -0.12767835, -0.582088  , -0.96807895, -1.60672381, -0.35158743,
       -0.20239202,  0.90029399,  0.61864161,  0.6552046 , -0.8876166 ,
        0.51014601, -0.39193716, -0.22991784, -0.53677412,  0.32106478,
        0.39070571,  0.33279655, -0.616993  ,  1.18605888,  0.69475163,
       -0.58355471, -0.13264168, -0.78688543, -0.09417141, -0.31

In [85]:
# Fit the score review to a model (takes time)
lin_feel_train_model = lin_model.fit(training)

In [86]:
# Save the model for later use
lin_feel_train_model.write().overwrite().save("/content/gdrive/MyDrive/Project3_BeerQuality/lin_feel_train.md")

In [87]:
# Read the model
loaded_lin_feel_train_model = LinearRegressionModel.load("/content/gdrive/MyDrive/Project3_BeerQuality/lin_feel_train.md")

In [88]:
# Transform the training data to peek at the predictions
lin_feel_df = loaded_lin_feel_train_model.transform(training)

In [89]:
lin_feel_df.show()

+------------+--------------------+------------------+
|feel_numeric|               model|        prediction|
+------------+--------------------+------------------+
|         1.0|[-0.1452391079316...|3.3194511219231497|
|         1.0|[-0.1360762032201...| 3.611262939454425|
|         1.0|[-0.1245115498701...|1.6921212228851625|
|         1.0|[-0.1112089413319...|  3.65255601946466|
|         1.0|[-0.1096023616691...|3.1776604365840595|
|         1.0|[-0.1067826888812...|3.4561946075998318|
|         1.0|[-0.0992016121461...| 3.262108680631359|
|         1.0|[-0.0905129771966...|2.8486217540864427|
|         1.0|[-0.0898339535608...|3.4487757867856716|
|         1.0|[-0.0831069555772...| 3.404353344307464|
|         1.0|[-0.0829146506434...| 3.836702495924664|
|         1.0|[-0.0806717465166...| 4.248175490975582|
|         1.0|[-0.0735591155427...|3.6150582013247035|
|         1.0|[-0.0698463073019...|3.4710629762621994|
|         1.0|[-0.0685070913528...| 3.215086885473001|
|         

In [90]:
# show a sample of the predictions
lin_feel_df.select('feel_numeric', 'prediction').show(truncate=False)

+------------+------------------+
|feel_numeric|prediction        |
+------------+------------------+
|1.0         |3.3194511219231497|
|1.0         |3.611262939454425 |
|1.0         |1.6921212228851625|
|1.0         |3.65255601946466  |
|1.0         |3.1776604365840595|
|1.0         |3.4561946075998318|
|1.0         |3.262108680631359 |
|1.0         |2.8486217540864427|
|1.0         |3.4487757867856716|
|1.0         |3.404353344307464 |
|1.0         |3.836702495924664 |
|1.0         |4.248175490975582 |
|1.0         |3.6150582013247035|
|1.0         |3.4710629762621994|
|1.0         |3.215086885473001 |
|1.0         |4.037404261510018 |
|1.0         |3.895646011049197 |
|1.0         |3.116561428250774 |
|1.0         |3.400022481926565 |
|1.0         |3.2559481110674358|
+------------+------------------+
only showing top 20 rows



In [91]:
gr = lin_feel_df.groupBy("feel_numeric").count()
gr.show()

+------------+------+
|feel_numeric| count|
+------------+------+
|         5.0| 45894|
|         2.5| 26583|
|         2.0| 16211|
|         3.0| 99151|
|        2.75|  6354|
|         1.5|  4523|
|        3.75| 79198|
|        1.75|   891|
|        4.25| 87813|
|         3.5|197614|
|         1.0|  4088|
|        2.25|  1811|
|        4.75| 25867|
|         4.5|171671|
|        1.25|   579|
|        3.25| 20821|
|         4.0|396447|
+------------+------+



In [92]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.4 gigabytes of available RAM

You are using a high-RAM runtime!
